In [2]:
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv('mutations_1.csv')
data = df

In [12]:
def split_data(data):
    split_1 = data.sample(frac=0.33, random_state=0)
    rem_data =data.drop(split_1.index)
    split_2 = rem_data.sample(frac=0.5, random_state=1)
    split_3 = rem_data.drop(split_2.index)
    
    return split_1, split_2, split_3 

In [18]:
split_1, split_2, split_3  = split_data(data)

In [21]:
train_1 = pd.concat([split_1, split_2], axis=0)
train_1

,class,GOT1_GRCh37_10:101163586-101163586_Missense-Mutation_SNP_C-C-T,TEX36_GRCh37_10:127371546-127371546_Nonsense-Mutation_SNP_G-G-A,KIAA1217_GRCh37_10:24810824-24810824_Missense-Mutation_SNP_C-C-T,SIRT1_GRCh37_10:69676051-69676051_Missense-Mutation_SNP_C-C-T,MYPN_GRCh37_10:69881845-69881845_Missense-Mutation_SNP_C-C-T,LARP4B_GRCh37_10:890939-890939_Frame-Shift-Del_DEL_T-T--,NPAT_GRCh37_11:108032094-108032094_Missense-Mutation_SNP_G-G-T_G-G-A,ARNTL_GRCh37_11:13402723-13402723_Silent_SNP_G-G-A,COPB1_GRCh37_11:14480092-14480092_Missense-Mutation_SNP_G-G-A_G-G-T,...,FBXO21_GRCh37_12:117624320-117624320_Frame-Shift-Del_DEL_A-A--,CLRN1_GRCh37_3:150644691-150644691_3'UTR_DEL_A-A--,VPS37A_GRCh37_8:17153149-17153149_3'UTR_DEL_A-A--,VANGL2_GRCh37_1:160398257-160398258_3'UTR_INS_----G,SRGAP3_GRCh37_3:9023972-9023972_3'UTR_DEL_A-A--,PTENP1_GRCh37_9:33674774-33674774_RNA_DEL_T-T--,KMT2C_GRCh37_7:151836804-151836804_Nonsense-Mutation_SNP_G-G-A,CDKN1C_GRCh37_11:2904846-2904846_3'UTR_DEL_T-T--,PIK3R2_GRCh37_19:18278070-18278070_Missense-Mutation_SNP_A-A-G,LMO7_GRCh37_13:76432354-76432354_3'UTR_SNP_C-C-T
225,NC123,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
122,C55,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
92,C41,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
157,NC86,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
154,C71,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,C51,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
93,C42,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
79,C34,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
134,NC75,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
test_1 = split_3
test_1

,class,GOT1_GRCh37_10:101163586-101163586_Missense-Mutation_SNP_C-C-T,TEX36_GRCh37_10:127371546-127371546_Nonsense-Mutation_SNP_G-G-A,KIAA1217_GRCh37_10:24810824-24810824_Missense-Mutation_SNP_C-C-T,SIRT1_GRCh37_10:69676051-69676051_Missense-Mutation_SNP_C-C-T,MYPN_GRCh37_10:69881845-69881845_Missense-Mutation_SNP_C-C-T,LARP4B_GRCh37_10:890939-890939_Frame-Shift-Del_DEL_T-T--,NPAT_GRCh37_11:108032094-108032094_Missense-Mutation_SNP_G-G-T_G-G-A,ARNTL_GRCh37_11:13402723-13402723_Silent_SNP_G-G-A,COPB1_GRCh37_11:14480092-14480092_Missense-Mutation_SNP_G-G-A_G-G-T,...,FBXO21_GRCh37_12:117624320-117624320_Frame-Shift-Del_DEL_A-A--,CLRN1_GRCh37_3:150644691-150644691_3'UTR_DEL_A-A--,VPS37A_GRCh37_8:17153149-17153149_3'UTR_DEL_A-A--,VANGL2_GRCh37_1:160398257-160398258_3'UTR_INS_----G,SRGAP3_GRCh37_3:9023972-9023972_3'UTR_DEL_A-A--,PTENP1_GRCh37_9:33674774-33674774_RNA_DEL_T-T--,KMT2C_GRCh37_7:151836804-151836804_Nonsense-Mutation_SNP_G-G-A,CDKN1C_GRCh37_11:2904846-2904846_3'UTR_DEL_T-T--,PIK3R2_GRCh37_19:18278070-18278070_Missense-Mutation_SNP_A-A-G,LMO7_GRCh37_13:76432354-76432354_3'UTR_SNP_C-C-T
1,C1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NC3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,NC9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,C4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,NC11,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,NC131,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
239,NC134,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
245,NC140,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
246,C107,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Decision Tree

In [24]:
def MostUsefulFeature(df): #get most useful feature with highest TP-FP
    diff={}

    for col in df.columns[1:]:
        TP = len(df[(df[col]==1) & (df.iloc[:,0].str.startswith('C'))])
        FP = len(df[(df[col]==1) & (df.iloc[:,0].str.startswith('NC'))])

        diff[col] = TP-FP
        
        
    # return [max(diff, key=diff.get), max(diff.values())]
    return max(diff, key=diff.get)

In [28]:
def tree(df):
    MUF = MostUsefulFeature(df)
    group_a = df[df[MUF]==1]
    group_b = df[df[MUF]==0]
    
    MUA = MostUsefulFeature(group_a)
    MUB = MostUsefulFeature(group_b)
    
    tree_list = [MUF, MUA, MUB] 
    return tree_list

In [30]:
tree_1 = tree(data)
tree_1

['RPL22_GRCh37_1:6257785-6257785_Frame-Shift-Del_DEL_T-T--',
 'RPL22_GRCh37_1:6257785-6257785_Frame-Shift-Del_DEL_T-T--',
 'DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-Del_DEL_C-C--']

In [37]:
def Classify(df:pd.DataFrame, sample:pd.Series, tree_list:list):
    # sample = df[df["class"]==sample]
    # sample = df.iloc[[sample]]
    if sample[tree_list[0]] == 1:
        if sample[tree_list[1]] == 1: 
            return "C"  
        else:
            return "NC"  
     
    elif sample[tree_list[0]] == 0:
        if sample[tree_list[2]] == 1: 
            return "C"  
        else:
            return "NC"  

In [79]:
def Classify_test(tree, test):
    my_map = {}
    for i in range(len(test)):
        check = test.iloc[i]['class']
        my_map[check] = Classify(test, test.iloc[i], tree)
   
    return my_map

In [80]:
test_1_map = Classify_test(tree_1, test_1)

In [81]:
def predicted_actual(test_map):
    test_df = pd.DataFrame(test_map, index=[0])
    test_df = test_df.transpose()
    test_df = test_df.reset_index()
    test_df.rename(columns={'index': 'ActualClass'}, inplace=True)
    test_df.rename(columns={0: 'PredictedClass'}, inplace=True)
    test_df['ActualClass'] = test_df['ActualClass'].str.startswith('C').astype(int)
    test_df['PredictedClass'] = test_df['PredictedClass'].str.startswith('C').astype(int)
    return test_df

In [82]:
pred_actual_df_1 = predicted_actual(test_1_map)

In [83]:
def ConfusionMatrix(pred_actual_bool:pd.DataFrame):
    TP, TN, FN, FP = 0, 0, 0, 0
    for actual, predicted in zip(pred_actual_bool['ActualClass'], pred_actual_bool['PredictedClass']):
        if actual == predicted:
            if actual == 1:
                TP += 1
            else:
                TN += 1
        else: 
            if actual == 1:
                FN += 1
            else:
                FP += 1 
            
        values = [TP, TN, FP, FN]
    return values

In [84]:
Conf_mat_1  = ConfusionMatrix(pred_actual_df_1)

In [109]:
def ClassificationMetrics(conf_mat):
    TP, TN, FP, FN = conf_mat
    Accuracy = (TP + TN)/(TP + TN + FP + FN)
    Sensitivity = (TP)/(TP + FN)
    Specificity = (TN)/(TN + FP)
    Precision = (TP)/(TP + FP)
    Miss_rate = (FN)/(TP + FN)
    False_Discovery_Rate = (FP)/(TP + FP)
    False_Omission_Rate = (FN)/(TN + FN)
    
    result = f"""Accuracy: {Accuracy*100: .2f}%. \n
    Sensitivity: {Sensitivity*100: .2f}%. \n
    Specificity: {Specificity*100: .2f}%. \n
    Precision: {Precision*100: .2f}%. \nMiss Rate: {Miss_rate*100: .2f}%. \n
    False Discovery Rate: {False_Discovery_Rate*100: .2f}%. \n
    False Ommission Rate: {False_Omission_Rate*100: .2f}%."""
    return result

In [115]:
print(ClassificationMetrics(Conf_mat_1))

Accuracy:  72.62%. 
Sensitivity:  43.59%. 
Specificity:  97.78%. 
Precision:  94.44%. 
Miss Rate:  56.41%. 
False Discovery Rate:  5.56%. 
False Ommission Rate:  33.33%.


In [97]:
def train_and_test(train, test, group):
    print(f"Starting Training and Testing for Group{group} \n")
    decision_tree = tree(train) #train decision tree with train set
    print("Decision Tree:", decision_tree, '\n')
    Classified_test = Classify_test(decision_tree, test)
    pred_actual_df = predicted_actual(Classified_test)
    Confusion_Matrix_values = ConfusionMatrix(pred_actual_df)
    print("Confusion_Matrix:", Confusion_Matrix_values, '\n')
    metrics = ClassificationMetrics(Confusion_Matrix_values)
    print(metrics)


In [112]:
train_A, test_A = pd.concat([split_1, split_2], axis=0), split_3
train_and_test(train_A, test_A, 'A')

Starting Training and Testing for GroupA 

Decision Tree: ['RPL22_GRCh37_1:6257785-6257785_Frame-Shift-Del_DEL_T-T--', 'RPL22_GRCh37_1:6257785-6257785_Frame-Shift-Del_DEL_T-T--', 'DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-Del_DEL_C-C--'] 

Confusion_Matrix: [17, 44, 1, 22] 

Accuracy:  72.62%. 
Sensitivity:  43.59%. 
Specificity:  97.78%. 
Precision:  94.44%. 
Miss Rate:  56.41%. 
False Discovery Rate:  5.56%. 
False Ommission Rate:  33.33%.


In [113]:
train_B, test_B = pd.concat([split_2, split_3], axis=0), split_1
train_and_test(train_B, test_B, 'B')

Starting Training and Testing for GroupB 

Decision Tree: ['RPL22_GRCh37_1:6257785-6257785_Frame-Shift-Del_DEL_T-T--', 'RPL22_GRCh37_1:6257785-6257785_Frame-Shift-Del_DEL_T-T--', 'RNF43_GRCh37_17:56435161-56435161_Frame-Shift-Del_DEL_C-C--'] 

Confusion_Matrix: [11, 48, 0, 23] 

Accuracy:  71.95%. 
Sensitivity:  32.35%. 
Specificity:  100.00%. 
Precision:  100.00%. 
Miss Rate:  67.65%. 
False Discovery Rate:  0.00%. 
False Ommission Rate:  32.39%.


In [116]:
train_C, test_C = pd.concat([split_1, split_3], axis=0), split_2
train_and_test(train_C, test_C, 'C')

Starting Training and Testing for GroupC 

Decision Tree: ['RNF43_GRCh37_17:56435161-56435161_Frame-Shift-Del_DEL_C-C--', 'RNF43_GRCh37_17:56435161-56435161_Frame-Shift-Del_DEL_C-C--', 'RPL22_GRCh37_1:6257785-6257785_Frame-Shift-Del_DEL_T-T--'] 

Confusion_Matrix: [8, 48, 1, 27] 

Accuracy:  66.67%. 
Sensitivity:  22.86%. 
Specificity:  97.96%. 
Precision:  88.89%. 
Miss Rate:  77.14%. 
False Discovery Rate:  11.11%. 
False Ommission Rate:  36.00%.
